In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

device = torch.device("cuda")
print(torch.cuda.is_available())
bertmodel, vocab = get_pytorch_kobert_model()

dataset_train = nlp.data.TSVDataset('./data/train_hate_dataset_v2.txt')#, field_indices=[1,2])
dataset_test = nlp.data.TSVDataset('./data/test_hate_dataset_v2.txt')#, field_indices=[1,2])

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

## Setting parameters
max_len = 32
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  6e-5 #5e-5

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=0)

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=1,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Sequential(
          nn.Linear(hidden_size , num_classes),
          nn.Sigmoid()
        )
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

model = BERTClassifier(bertmodel,  dr_rate=0.5)
#멀티GPU 사용 시.
#if torch.cuda.device_count() > 1:
#    model =  nn.DataParallel(model)
model.to(device)
print(model)
#model.to(f'cuda:{model.device_ids[0]}')
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
#loss_fn = nn.CrossEntropyLoss()
loss_fn = nn.BCELoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

best_f1 = 0.0

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    y_pred, y_true = [], []
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.float().to(device)
        out = model(token_ids, valid_length, segment_ids).squeeze()
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        pred = (out>0.5).to(torch.float)
        y_pred.extend(pred)
        y_true.extend(label)
    y_true_cpu = [int(x) for x in y_true]
    y_pred_cpu = [int(x) for x in y_pred]
    correct = sum([(x==y) for x,y in zip(y_pred, y_true)])
    precision = precision_score(y_true_cpu, y_pred_cpu)
    recall = recall_score(y_true_cpu, y_pred_cpu)
    f1 = f1_score(y_true_cpu, y_pred_cpu)
    accuracy = correct / (len(y_pred))
    print("epoch {} train accuracy {:.3f} f1_score {:.3f} precision {:.3f} recall {:.3f}".format(e+1, accuracy, f1, precision, recall))
    model.eval()
    y_true, y_pred = [], []
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.float().to(device)
        out = model(token_ids, valid_length, segment_ids).squeeze()
        pred = (out>0.5).to(torch.float)
        y_pred.extend(pred)
        y_true.extend(label)
    y_true_cpu = [int(x) for x in y_true]
    y_pred_cpu = [int(x) for x in y_pred]
    correct = sum([(x==y) for x,y in zip(y_pred, y_true)])
    precision = precision_score(y_true_cpu, y_pred_cpu)
    recall = recall_score(y_true_cpu, y_pred_cpu)
    f1 = f1_score(y_true_cpu, y_pred_cpu)
    accuracy = correct / (len(y_pred))
    print("epoch {} test accuracy {:.3f} f1_score {:.3f} precision {:.3f} recall {:.3f}".format(e+1, accuracy, f1, precision, recall))
    
    if f1 > best_f1:
        best_model = model
        best_f1 = f1
        torch.save(best_model.state_dict(), './check_point/koBERT_state.pt')


c:\Users\Safwan\AppData\Local\Programs\Python\Python37\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


True
using cached model. c:\Users\Safwan\Desktop\Profanity\nlp-filter-out\.cache\kobert_v1.zip
using cached model. c:\Users\Safwan\Desktop\Profanity\nlp-filter-out\.cache\kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. c:\Users\Safwan\Desktop\Profanity\nlp-filter-out\.cache\kobert_news_wiki_ko_cased-1087f8699e.spiece
BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
   

c:\Users\Safwan\AppData\Local\Programs\Python\Python37\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
c:\Users\Safwan\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:119: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/324 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [1]:
best_f1

NameError: name 'best_f1' is not defined